# Guía 3 — Introducción a LangChain con OpenAI API
Curso: **IA en el Aula — Nivel Avanzado**
Profesor: **Luis Daniel Benavides Navarro**
Fecha: **Octubre 2025**
Estudiantes: **David Alfonso Barbosa Gómez**

En esta guía aprenderás los conceptos básicos de **LangChain**, una librería diseñada para construir aplicaciones impulsadas por modelos de lenguaje, integrando la API de OpenAI con flujos más complejos. Exploraremos cómo crear un primer **prompt chain**, administrar memoria y usar herramientas para componer respuestas inteligentes.

## 1️⃣ Instalación y configuración inicial
LangChain se instala como cualquier otra librería de Python. También usaremos `python-dotenv` para gestionar la clave de OpenAI. Ejecute la siguiente celda:

In [1]:
%pip install openai langchain python-dotenv langchain-openai


Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install langchain langchain-core langchain-openai openai python-dotenv

Note: you may need to restart the kernel to use updated packages.


2️⃣ Cargar variables de entorno y cliente OpenAI
Crea un archivo .env con tu clave de API y cárgala para poder usarla dentro de LangChain. Esto evita exponer tu clave en el código fuente.

In [8]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv()  # Cargar archivo .env
api_key = os.getenv("OPENAI_API_KEY")

if not api_key:
    raise ValueError("No se encontró la clave OPENAI_API_KEY en el archivo .env")

# Crear cliente LangChain con OpenAI
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.5)
print("Cliente LangChain con OpenAI inicializado correctamente.")

Resultado final:
 **Aplicación Educativa: "ExplorAR: Aventura de Aprendizaje Aumentado"**

**Descripción de la Aplicación:**
ExplorAR es una aplicación educativa de realidad aumentada diseñada para enriquecer el aprendizaje en diversas materias, como ciencias, historia, geografía y matemáticas. La aplicación permite a los estudiantes interactuar con elementos virtuales superpuestos en su entorno real, facilitando una experiencia de aprendizaje inmersiva y práctica.

**Características Principales:**

1. **Exploración de Contenidos:**
   - Los estudiantes pueden escanear objetos o lugares en su entorno (por ejemplo, un libro de texto, un edificio histórico o una planta) y la aplicación mostrará información adicional en forma de texto, imágenes, videos o modelos 3D. Por ejemplo, al apuntar su dispositivo a un libro de biología, pueden ver un modelo 3D del sistema circulatorio que pueden rotar y explorar.

2. **Actividades Interactivas:**
   - La aplicación incluye juegos y desafíos intera

## 3️⃣ Primer ejemplo: Prompt simple con LangChain
LangChain utiliza **chains** (cadenas de pasos) para procesar información. Comencemos con una cadena simple que envía un mensaje al modelo y obtiene la respuesta.

In [9]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Initialize the LLM (uses your OpenAI API key from environment)
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.5)

# Create a simple prompt
prompt = ChatPromptTemplate.from_template(
    "Explica en dos frases el concepto de {tema}."
)

# Combine the components using LCEL (LangChain Expression Language)
chain = prompt | llm | StrOutputParser()

# Run it
result = chain.invoke({"tema": "aprendizaje automático"})
print(result)


El aprendizaje automático es una rama de la inteligencia artificial que permite a las computadoras aprender de datos y mejorar su rendimiento en tareas específicas sin ser programadas explícitamente para cada una de ellas. Utiliza algoritmos que identifican patrones y hacen predicciones basadas en la información proporcionada.


### Explicación
- **ChatPromptTemplate:** define la estructura del mensaje que se envía al modelo.
- **ChatOpenAI:** la conexión real al modelo.
- **chain** crea la cadena de componentes usando LCEL (LangChain Expression Language).
- **StrOutputParser** convierte la salida estructurada del modelo en texto plano.
- **chain.invoke:** ejecuta la cadena pasando los valores del prompt.

Este patrón permite reutilizar prompts para distintos temas o contextos.

## 4️⃣ Ejemplo 2: Encadenar múltiples pasos
LangChain permite combinar varios pasos en una misma ejecución. En este ejemplo, crearemos dos prompts: uno para definir un tema y otro para generar una aplicación educativa basada en el resultado.

In [11]:
# Cargar API key
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("No se encontró la clave OPENAI_API_KEY en el archivo .env")

# Inicializar modelo
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.5)

# Primer paso: obtener una descripción
primer_prompt = ChatPromptTemplate.from_template(
    "Explica brevemente el concepto de {tema}."
)
primer_chain = primer_prompt | llm | StrOutputParser()

# Segundo paso: generar aplicación educativa
segundo_prompt = ChatPromptTemplate.from_template(
    "Propón una aplicación educativa del siguiente concepto: {concepto}."
)
segundo_chain = segundo_prompt | llm | StrOutputParser()

concepto = primer_chain.invoke({"tema": "realidad aumentada"})
resultado = segundo_chain.invoke({"concepto": concepto})

print("Resultado final:\n")
print(resultado)

Resultado final:

**Aplicación Educativa: "Exploradores del Conocimiento"**

**Descripción General:**
"Exploradores del Conocimiento" es una aplicación educativa de realidad aumentada diseñada para estudiantes de educación primaria y secundaria. Su objetivo es enriquecer el aprendizaje en diversas materias, como ciencias, historia y geografía, mediante la superposición de contenido digital interactivo sobre el entorno real del usuario.

**Características de la Aplicación:**

1. **Exploración de Entornos:**
   - Los estudiantes pueden escanear su entorno (aula, casa, parque) con la cámara de su dispositivo. La aplicación reconoce el espacio y ofrece actividades interactivas basadas en el lugar escaneado.
   - Por ejemplo, al escanear un mapa del mundo, los estudiantes pueden ver información sobre diferentes países, incluyendo datos culturales, históricos y geográficos.

2. **Lecciones Interactivas:**
   - La aplicación ofrece lecciones en formato de RA sobre temas específicos. Al apunta

## 5️⃣ Ejemplo 3: Añadir memoria a la conversación
LangChain incluye módulos de **memoria** para mantener contexto entre múltiples interacciones. Esto permite simular conversaciones educativas más naturales, donde el modelo recuerda temas previos.

In [12]:
%pip install -U langchain langchain-openai

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.output_parsers import StrOutputParser

# LLM (requiere OPENAI_API_KEY en el entorno)
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.5)
to_str = StrOutputParser()

# Prompt con hueco para el historial
prompt = ChatPromptTemplate.from_messages([
    ("system", "Eres un asistente educativo claro y conciso."),
    MessagesPlaceholder("chat_history"),      # ← aquí va la memoria
    ("human", "{input}")
])

# Cadena base
chain = prompt | llm | to_str

# Memoria simple como lista de mensajes
history: list = []

def chat(user_text: str) -> str:
    """
    Envía un turno del usuario, usa el historial y actualiza la memoria
    con el par (usuario, asistente).
    """
    global history
    # Ejecutar la cadena inyectando el historial actual
    answer = chain.invoke({"input": user_text, "chat_history": history})
    # Actualizar memoria (guardar los dos mensajes)
    history += [HumanMessage(content=user_text), AIMessage(content=answer)]
    return answer

# --- Ejemplo de uso (tres turnos) ---
print(chat("Hola, soy un profesor de informática."))
print(chat("¿Puedes explicarme cómo introducir IA a mis estudiantes?"))
print(chat("¿Qué ejemplos prácticos puedo usar en la clase?"))

Note: you may need to restart the kernel to use updated packages.
¡Hola! Encantado de ayudarte. ¿En qué puedo asistirte hoy en tu labor como profesor de informática?
Claro, aquí tienes algunas sugerencias para introducir la inteligencia artificial (IA) a tus estudiantes:

1. **Conceptos Básicos**: Comienza con una introducción a los conceptos fundamentales de la IA, como qué es, su historia y las diferencias entre IA débil y fuerte.

2. **Aplicaciones Prácticas**: Muestra ejemplos de cómo se utiliza la IA en la vida cotidiana, como asistentes virtuales (Siri, Alexa), recomendaciones de productos (Netflix, Amazon) y reconocimiento facial.

3. **Herramientas y Plataformas**: Introduce herramientas accesibles para aprender sobre IA, como Google Teachable Machine, Scratch con extensiones de IA, o plataformas de programación como Python con bibliotecas como TensorFlow o PyTorch.

4. **Proyectos Prácticos**: Diseña proyectos sencillos donde los estudiantes puedan crear sus propios modelos de

Explicación
ConversationBufferMemory: almacena los mensajes anteriores en la conversación.
ConversationChain: combina el modelo con la memoria.
Esta funcionalidad es útil para tutores inteligentes o chatbots educativos que requieren continuidad en el diálogo.

## 6️⃣ Buenas prácticas con LangChain + OpenAI
- Usa prompts **claros y estructurados**: el modelo responde mejor cuando la tarea está bien definida.
- Controla `temperature` según la tarea: bajo (0.1–0.3) para precisión, alto (0.7–0.9) para creatividad.
- Guarda logs o historiales si tu aplicación incluye interacción prolongada.
- Limita la longitud de las respuestas con `max_tokens` para evitar costos o respuestas excesivas.
- Documenta tus cadenas (`LLMChain`) para reusarlas en distintos contextos educativos.

## ✅ Conclusión
Has aprendido los fundamentos de LangChain: prompts, chains, memoria y flujo secuencial. Estos conceptos son la base para desarrollar asistentes educativos, tutores personalizados o sistemas de generación de contenido en el aula. En la próxima guía implementaremos un **asistente educativo con RAG (Retrieval-Augmented Generation)** usando tus propios materiales docentes.